# Import library

In [34]:
import requests
import json
import pandas as pd
import numpy as np

# Helper function for REST call : GET / PUT / POST /DELETE

In [76]:
import requests
import json
import pandas as pd
#helper
def get(url):
    """REST CALL : GET"""
    reponse = requests.get(url, verify=False)
    if (reponse.ok):
        val = reponse.json()
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def delete(url ):
    """REST CALL : DELETE"""
    reponse = requests.delete(url,verify=False)
    if (reponse.ok):
        val = "data deleted"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def put(url, json):
    """REST CALL : PUT"""
    reponse = requests.put(url, json=json, verify=False)
    if (reponse.ok):
        val = "build"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val

def post(url, json,isCsv=False):
    """REST CALL : POST"""
    reponse = requests.post(url, json=json, verify=False,headers={'Connection':'close'})
    valPD = pd.DataFrame(columns=['id',"npv"])
    if(reponse.ok):
        if(reponse.status_code==201):
            val = reponse.headers._store['location'][1]
        else:
            if isCsv:
                lines = reponse.text.splitlines()
                val=[]
                ids=[]
                npvs=[]
                for item in range(1,len(lines)):
                    id,npv = lines[item].split(";")
                    val.append({"id" : id,"npv" : npv})
                    ids.append(id)
                    npvs.append(npv)
                valPD["id"]=ids
                valPD["npv"] = npvs
            else:
                val = reponse.json()
    elif(reponse.status_code==400):
        val= reponse.text
    else:
        status = reponse.raise_for_status()
        val =  "error : " + status
    reponse.close()
    return val , valPD



In [82]:
url_fcp_swagger = "https://fr1pslcmf05:8770/api/"
url_pricing_config = "https://fr1pslcmf05:8770/api/pricing/configs"
url_result_handler ="https://fr1pslcmf05:8770/api/pricing/report/result/Pricing"

def unitary(configName,aod,pricingMethod,marketDataSetId,resultHandlerId,referenceCurrency,resultHandlerConfigId,marketDataProviderId,perimeter,scenarioContexts):
    unitary_price = {}
    unitary_price["asOfDate"] = aod
    unitary_price["referenceCurrency"] = referenceCurrency     
    unitary_price["pricingMethod"] = pricingMethod
    unitary_price["marketDataSetId"] = marketDataSetId
    unitary_price["resultHandlerId"] = resultHandlerId
    unitary_price["resultHandlerConfigId"] = resultHandlerConfigId
    unitary_price["marketDataProviderId"] = marketDataProviderId    
    unitary_price["pricingConfigId"] = getPricingId(configName)
    unitary_price["perimeter"] = {"trade" : {"ids": ["FXOptionVanilla_1"]}}
    unitary_price["scenarioContexts"] = scenarioContexts
    url_price = url_fcp_swagger + "pricing/price/"
    post(url=url_price, json=unitary_price)
    return getResult()

def getResult(id="Pricing",dimensions="Id",measures="base_NPV"):
    url="https://fr1pslcmf05:8770/api/pricing/report/result/"+id
    resultHandler={}
    resultHandler["dimensions"]= [dimensions]
    resultHandler["measures"]= [measures]
    return post(url,json=resultHandler,isCsv=True)

scenarioContexts =[

    {
      "id": "base",
      "measureGroupIds": [
        "NPV"
      ]
    }

]

def getPricingId(configName):
    urlConfig = "https://fr1pslcmf05:8770/api/pricing/configs"
    config = get(urlConfig)
    for item in config:
        if (item['name']==configName):
            return item["id"]
    return "error"

# NPV use case

In [87]:
resultJson, resultPD = unitary(configName="DEFAULT",aod="2016-07-04",pricingMethod="THEORETICAL",marketDataSetId="$id/DEFAULT",referenceCurrency="$id/USD",marketDataProviderId="PE_STORE_MDP",resultHandlerConfigId="Default",perimeter="ALL_FX_OPTION", scenarioContexts = scenarioContexts,resultHandlerId="Report")

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [88]:
resultPD

,id,npv
0,FXOptionVanilla_1,-6.847514388329348E-13


# NPV FOR ONE TRADE